In [5]:
import yfinance as yf
import pandas as pd
import random
import time
import networkx as nx
import json
from pyvis.network import Network

class Stock:
    def __init__(self, symbol, name, sector="Unknown"):
        self.symbol = symbol
        self.name = name
        self.sector = sector
        self.k_score = 0

# Fetch the list of S&P 500 components
snp500_components = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
all_stock_symbols = snp500_components['Symbol'].tolist()

# Set the random seed from the current time
random.seed(int(time.time()))
n = 30
# Randomly choose 100 stocks from the S&P 500 components
selected_stock_symbols = random.sample(all_stock_symbols, n)

# Remove specific stocks if present and add replacements
stock_replacements = {
    "BF.B": "BF-B",
    "BRK.B": "BRK-B",
    "KVUE": None,
    "GEHC": None,
    "NOW": None,
    "CI": None
}
selected_stock_symbols = [stock_replacements.get(symbol, symbol) for symbol in selected_stock_symbols if stock_replacements.get(symbol, symbol)]

# Fetch sector information for the selected stocks
sectors = {}
for symbol in selected_stock_symbols:
    stock_info = yf.Ticker(symbol).info
    sector = stock_info.get('sector', "Unknown")
    sectors[symbol] = sector

# Fetch historical stock price data from Yahoo Finance for the selected stocks
end_date = pd.Timestamp.now().normalize()  # Today's date
start_date = end_date - pd.DateOffset(days=90)  # 90 days ago

# Fetch minute-level stock price data from Yahoo Finance for the previous trading day (yesterday)
stock_prices = yf.download(selected_stock_symbols, start=start_date, end=end_date, interval="5d")["Adj Close"]

# Create a list of Stock objects with initialized k_score and sector
stock_objects = [Stock(symbol=symbol, name=yf.Ticker(symbol).info.get('longName', "No Name"), sector=sectors[symbol]) for symbol in selected_stock_symbols]

# Calculate correlations between stock returns
stock_returns = stock_prices.pct_change().dropna()
correlation_matrix = stock_returns.corr()

# Create a weighted financial network using NetworkX
G = nx.Graph()

# Get the top 5% correlations (excluding NaN and self-correlations)
top_correlations = (
    correlation_matrix.unstack()
    .sort_values(ascending=False)
    .dropna()
    .reset_index()
    .rename(columns={"Ticker": "stock1", "Ticker": "stock2", 0: "correlation"})
    .query("stock1 != stock2")
    .head(int(min(400,n*(n-1)/2*0.20)))
)


# Calculate k_score for each stock object and update the k_score attribute
for row in top_correlations.itertuples(index=False):
    stock1, stock2, correlation = row
    for stock_obj in stock_objects:
        if stock_obj.symbol == stock1 or stock_obj.symbol == stock2:
            stock_obj.k_score += correlation

# Add nodes to the graph using stock objects and color them based on sectors
sector_colors = {}
for stock_obj in stock_objects:
    if stock_obj.k_score > 0:
        sector = stock_obj.sector
        # Assign a color if the sector is new, otherwise use the existing color
        if sector not in sector_colors:
            sector_colors[sector] = '#' + ''.join(random.choice('0123456789ABCDEF') for _ in range(6))
        G.add_node(
            stock_obj.symbol,
            label=stock_obj.name,
            k_score=stock_obj.k_score,
            color=sector_colors[stock_obj.sector],
            sector=sector
        )

# Add weighted edges with top correlations as labels
for row in top_correlations.itertuples(index=False):
    stock1, stock2, correlation = row
    if stock1 != stock2 and stock1 in G.nodes() and stock2 in G.nodes():
        G.add_edge(stock1, stock2, weight=correlation)

# Convert NetworkX graph to PyVis graph
net = Network(height="800px", width="1600px", notebook=True)
for node in G.nodes:
    attributes = G.nodes[node]
    net.add_node(node, label=attributes['label'], k_score=attributes['k_score'], color=attributes['color'])
for edge in G.edges:
    attributes = G.get_edge_data(*edge)
    net.add_edge(edge[0], edge[1], value=attributes['weight'], title=f"Correlation: {attributes['weight']:.2f}")

""""
# Visualize the financial network using PyVis
net.show("financial_network.html")

# Create a legend for sector colors
legend_html = '<div style="background-color:white; padding:10px; border:1px solid #ccc; position:absolute; top:10px; right:10px; z-index:1000;">'
for sector, color in sector_colors.items():
    legend_html += f'<div style="margin-bottom:5px;"><div style="width:20px; height:20px; background-color:{color}; display:inline-block;"></div><span>{sector}</span></div>'
legend_html += '</div>'

# Open the generated HTML file for appending
with open("financial_network.html", "a") as html_file:
    # Inject the legend HTML into the generated HTML file
    html_file.write(legend_html)

"""
# Create an empty list to store the edge data in the desired format
edge_data = []

# Add edges with attributes such as value and title
for edge in G.edges:
    attributes = G.get_edge_data(*edge)
    edge_item = {
        "from": edge[0],
        "to": edge[1],
        "value": attributes['weight'],
        "title": f"Correlation: {attributes['weight']:.2f}"
    }
    edge_data.append(edge_item)

# Convert the list of edge data to a JSON-formatted string
edges_json = json.dumps(edge_data)




node_data = []

# Add nodes with labels, k_score, color, and other attributes
for node in G.nodes:
    attributes = G.nodes[node]
    node_item = {
        "color": attributes['color'],
        "id": node,
        "k_score": attributes['k_score'],
        "label": attributes['label'],
        "sector": attributes['sector'],
        "shape": "dot"  # You can set the shape here
    }
    node_data.append(node_item)

sector_data = []
print (sectors)
for sector in sectors:
    secotr_item = {
        "sector": attributes['sector'],
        "color": attributes['color']
    }
    sector_data.append(sector_data)

sector_colors_json = json.dumps(sector_colors)

# Convert the list of node data to a JSON-formatted string
nodes_json = json.dumps(node_data)


with open("sectorColoursJSON.js", "w") as sector_colors_json:
    sector_colors_json.write(f'var sectorColours = {sector_colors_json};')

with open("nodesJSON.js", "w") as nodes_file:
    nodes_file.write(f'var nodesJSON = {nodes_json};')

with open("edgesJSON.js", "w") as edges_file:
    edges_file.write(f'var edgesJSON = {edges_json};')


[*********************100%***********************]  30 of 30 completed


ValueError: cannot insert Ticker, already exists